In [1]:
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic

adata = sc.read_h5ad('withDRpseudotime_process.h5ad')

In [6]:
from scipy.interpolate import make_interp_spline
plt.rcParams['svg.fonttype'] = 'none'

gene = 'PXDN'
te_data = adata[:, gene].layers['TE'].flatten()
dr_data = adata[:, gene].layers['DR_All'].flatten()
pseudotime = adata.obs['Pseudotime']
n_bins = 20
bins = np.linspace(pseudotime.min(), pseudotime.max(), n_bins + 1)

te_means = np.zeros(n_bins)
dr_means = np.zeros(n_bins)

for i in range(n_bins):
    mask = (pseudotime >= bins[i]) & (pseudotime < bins[i+1])
    bin_cells = np.where(mask)[0]
    if len(bin_cells) > 0:
        te_means[i] = np.mean(te_data[bin_cells])
        dr_means[i] = np.mean(dr_data[bin_cells])

x_new = np.linspace(bins[:-1].min(), bins[:-1].max(), 300)
te_smooth = make_interp_spline(bins[:-1], te_means)(x_new)
dr_smooth = make_interp_spline(bins[:-1], dr_means)(x_new)

fig, ax1 = plt.subplots(figsize=(4, 4))
ax2 = ax1.twinx()

ax1.set_box_aspect(1)
ax2.set_box_aspect(1)

ax1.plot(x_new, te_smooth, color='red', label='TE', linewidth=0.5)
ax2.plot(x_new, dr_smooth, color='blue', label='TotalRNA', linewidth=0.5)

ax1.set_xlabel('Pseudotime')
ax1.set_ylabel('TE', color='red', fontsize=12)
ax2.set_ylabel('DR', color='blue', fontsize=12)
plt.title('PXDN Expression in Pseudotime', fontsize=14)

ax1.tick_params(axis='y', labelcolor='red', labelsize=10)
ax2.tick_params(axis='y', labelcolor='blue', labelsize=10)
ax1.tick_params(axis='x', labelsize=10)

ax1.grid(False)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()

plt.tight_layout()
plt.savefig('PXDN_expression_pseudotime.svg', format='svg')
plt.close()
